In [2]:
import pandas as pd
import numpy as np

#test_books = pd.read_csv('local_data/book_test.csv')
test_mails = pd.read_csv('local_data/mail_test.csv')
test_blogs = pd.read_csv('local_data/blog_test.csv')

In [2]:
test_books

NameError: name 'test_books' is not defined

In [3]:
test_mails

,id,pretokenized_text,decoded_text
0,mail_101,"[0, 5579, 38844, 28784, 19192, 50118, 45463, 1...",<s>--____________________________Next\nContent...
1,mail_101,"[175, 73, 40822, 73, 23008, 4, 47753, 116, 245...",com/links/click.cgi?5&3&1211134&/links/2/42/5/...
2,mail_101,"[6, 50118, 463, 47, 189, 253, 62, 15, 1365, 20...",",\nand you may end up on easy street.\nhttp://..."
3,mail_101,"[245, 73, 176, 73, 11762, 7485, 1694, 26378, 5...",5/2/calssweepsfreetext.cgi\nQ&A contest with C...
4,mail_101,"[10554, 18997, 37386, 21794, 46479, 50118, 491...","SportsLine Rewards Newsletter"">\n</head>\n<bod..."
...,...,...,...
67487,mail_91,"[1181, 28, 1051, 4, 50118, 4148, 302, 6, 550, ...","longer be sent.\nOn Monday, July 9, 2001 afte..."
67488,mail_91,"[111, 3506, 45, 111, 1549, 4, 1437, 1578, 5, 4...",-65 not -16. Also the ancillary\nis not addi...
67489,mail_91,"[1215, 37049, 36497, 6034, 5457, 163, 10259, 4...",_NOTIFICATION = BILL.WILLIAMS.III@ENRON.COM\nS...
67490,mail_91,"[3762, 410, 631, 15, 5, 564, 212, 4, 1437, 20,...",One little thing on the 25th. The ancillary a...


In [4]:
test_blogs

,id,pretokenized_text,decoded_text,age,topic,gender
0,blog_10009,"[0, 2387, 78, 3555, 4, 25980, 42, 66, 7, 192, ...",<s>My first entry. Testing this out to see how...,24,indUnk,male
1,blog_10009,"[7, 1004, 314, 23, 41, 8088, 14, 56, 10, 1109,...",to turn left at an intersection that had a li...,24,indUnk,male
2,blog_10009,"[1437, 1437, 1437, 1437, 1437, 1437, 1437, 143...",for the ungrateful.</s>,24,indUnk,male
3,blog_10017,"[0, 108, 100, 524, 10, 23611, 9376, 1437, 7774...",<s>'I am a banana!' Peace out.<\s>I haven't b...,15,Student,female
4,blog_10017,"[35, 996, 3326, 4, 520, 132, 1187, 675, 2012, ...",:15 AM. When 2nd period starts. All day we did...,15,Student,female
...,...,...,...,...,...,...
33698,blog_9989,"[79, 708, 7, 972, 42, 2173, 77, 52, 1108, 62, ...",she plans to meet this guy when we break up. ...,17,Student,male
33699,blog_9989,"[7735, 19, 256, 10708, 8, 38, 4, 166, 202, 192...",weird with Mandy and I. We still see each oth...,17,Student,male
33700,blog_9989,"[4, 166, 40, 192, 4, 1437, 38, 300, 127, 33734...",. We will see. I got my aquarium stuff the ot...,17,Student,male
33701,blog_9989,"[38, 40, 1874, 960, 8, 283, 878, 4, 404, 47, 3...",I will drop everything and come running. All ...,17,Student,male


In [6]:
# Utilities
#!pip install --upgrade scikit-learn

from sklearn.metrics import top_k_accuracy_score, accuracy_score
from transformers import AutoTokenizer
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
from random import shuffle

TOKENIZER = AutoTokenizer.from_pretrained('roberta-large')

def embed(model, texts):
    tokenized_texts = TOKENIZER(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    embedding = model(tokenized_texts.input_ids.to(model.device),
                      tokenized_texts.attention_mask.to(model.device),
                      )
    return embedding

def embed_transformer(model, texts):
    tokenized_texts = TOKENIZER(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    embedding = model(tokenized_texts.input_ids.to(model.device),
                      attention_mask = tokenized_texts.attention_mask.to(model.device),
                      ).pooler_output
    return embedding

def evaluate(model, data, top_k=5, N=100, repetitions=1, embed_f=embed):
    with torch.no_grad():
        
        accs, topks = [], []
        for _ in tqdm(range(repetitions)):           
            authors = data.id.unique().tolist()
            shuffle(authors)
            random_authors = authors[:N]
            anchors, replicas = [], []
            for author in random_authors:
                anchor, replica = data.loc[author == data.id].decoded_text.sample(2).tolist()
                anchors.append(anchor)
                replicas.append(replica)
            
            embedding_anchors = F.normalize(embed_f(model, anchors))
            embedding_replicas = F.normalize(embed_f(model, replicas))

            preds = embedding_anchors @ embedding_replicas.T
            labels = torch.arange(0, len(preds)).numpy()

            preds_a = F.softmax(preds, dim=-1)
            preds_b = F.softmax(preds.T, dim=-1)

            a_acc = accuracy_score(labels, preds_a.argmax(-1).cpu().numpy())
            b_acc = accuracy_score(labels, preds_b.argmax(-1).cpu().numpy())
            a_topk = top_k_accuracy_score(y_true=labels, y_score=preds_a.cpu().numpy(), k=top_k)
            b_topk = top_k_accuracy_score(y_true=labels, y_score=preds_b.cpu().numpy(), k=top_k)

            accs.append((a_acc+b_acc)/2)
            topks.append((a_topk+b_topk)/2)

            del embedding_anchors
            del embedding_replicas

            torch.cuda.empty_cache()
            
        return np.mean(accs), np.mean(topks), np.std(accs), np.std(topks)

/home/josephine/anaconda3/envs/sklearn-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
model_zoo = {#'all': 'model/final_2022-06-21_12-22-26_lstm_books+mails+blogs.ckpt',
            #'books': 'model/final_2022-06-28_07-55-16_lstm_books.ckpt',
            'mails': 'model/final_2022-06-27_08-14-08_lstm_mails.ckpt',
            'blogs': 'model/final_2022-06-15_08-10-17_lstm_blogs.ckpt',
            }
data_zoo = {#'all': test_books.append(test_mails).append(test_blogs),
            #'books': test_books,
            'mails': test_mails,
            'blogs': test_blogs,
            }

In [13]:
from model import ContrastiveTransformer
from random import shuffle

REPEATS = 100
TOP_K = 5
DEVICE = 1

n_list = [10, 20, 50, 100, 250]
#keys = ['all', 'blogs', 'books', 'mails']
keys = ['blogs', 'mails']

data_dict = {}
# Pass on trained models
for k_model in keys:
    model_ckpt = model_zoo[k_model]
    model = ContrastiveTransformer.load_from_checkpoint(checkpoint_path=model_ckpt).cuda(DEVICE)

    data_dict[f'Model_{k_model}'] = {}
    print(f'[log] Evaluating model - {k_model}')

    for k_data in keys:
        data = data_zoo[k_data]
        
        data_dict[f'Model_{k_model}'][f'data_{k_data}'] = {}
        print(f'[log] Evaluating on data - {k_data}')

        for n in n_list:
            print(f'[log] Running {REPEATS} repetitions for N={n}')
            max_len = len(data.id.unique())
            
            if n == 'max':
                n = max_len

            if n <= max_len:
                acc, topk, acc_sd, topk_sd = evaluate(model, data, top_k=TOP_K, N=n, repetitions=REPEATS)
                data_dict[f'Model_{k_model}'][f'data_{k_data}'][f'N={n} Accuracy'] = f'{100*acc:.2f}% ± {100*acc_sd:.2f}'
                data_dict[f'Model_{k_model}'][f'data_{k_data}'][f'N={n} Top-5 Accuracy'] = f'{100*topk:.2f}% ± {100*topk_sd:.2f}'

                print(f'[log] model:{k_model}_data:{k_data}_n:{n}: {100*acc:.2f}% {100*topk:.2f}%')
    
    del model
    torch.cuda.empty_cache()


model = AutoModel.from_pretrained('roberta-large').cuda(DEVICE)

data_dict[f'Model_RoBERTa'] = {}
print(f'[log] Evaluating model - RoBERTa')
#Base transformer
for k_data in keys:
    data = data_zoo[k_data]
        
    data_dict[f'Model_{k_model}'][f'data_{k_data}'] = {}
    print(f'[log] Evaluating on data - {k_data}')

    for n in n_list:
        print(f'[log] Running {REPEATS} repetitions for N={n}')
        max_len = len(data.id.unique())
        
        if n == 'max':
            n = max_len

        if n <= max_len:
            acc, topk, acc_sd, topk_sd = evaluate(model, data, top_k=TOP_K, N=n, repetitions=REPEATS, embed_f=embed_transformer)
            data_dict[f'Model_{k_model}'][f'data_{k_data}'][f'N={n} Accuracy'] = f'{100*acc:.2f}% ± {100*acc_sd:.2f}'
            data_dict[f'Model_{k_model}'][f'data_{k_data}'][f'N={n} Top-5 Accuracy'] = f'{100*topk:.2f}% ± {100*topk_sd:.2f}'

            print(f'[log] model:{k_model}_data:{k_data}_n:{n}: {100*acc:.2f}% {100*topk:.2f}%')


FileNotFoundError: [Errno 2] No such file or directory: '/home/josephine/proj/auth_ident/authorship-embeddings/model/final_2022-06-15_08-10-17_lstm_blogs.ckpt'

In [10]:
pd.DataFrame(data_dict)

,Model_all,Model_blogs,Model_books,Model_mails
data_all,"{'N=10 Accuracy': '91.25% ± 7.69', 'N=10 Top-5...","{'N=10 Accuracy': '90.35% ± 8.67', 'N=10 Top-5...","{'N=10 Accuracy': '58.25% ± 13.16', 'N=10 Top-...","{'N=10 Accuracy': '25.40% ± 11.48', 'N=10 Top-..."
data_blogs,"{'N=10 Accuracy': '90.60% ± 8.90', 'N=10 Top-5...","{'N=10 Accuracy': '91.55% ± 8.21', 'N=10 Top-5...","{'N=10 Accuracy': '52.60% ± 13.59', 'N=10 Top-...","{'N=10 Accuracy': '21.30% ± 10.38', 'N=10 Top-..."
data_books,"{'N=10 Accuracy': '87.75% ± 10.08', 'N=10 Top-...","{'N=10 Accuracy': '64.15% ± 12.71', 'N=10 Top-...","{'N=10 Accuracy': '83.50% ± 11.15', 'N=10 Top-...","{'N=10 Accuracy': '26.90% ± 12.22', 'N=10 Top-..."
data_mails,"{'N=10 Accuracy': '28.65% ± 9.79', 'N=10 Top-5...","{'N=10 Accuracy': '26.25% ± 11.76', 'N=10 Top-...","{'N=10 Accuracy': '21.80% ± 10.62', 'N=10 Top-...","{'N=10 Accuracy': '24.90% ± 12.43', 'N=10 Top-..."


In [19]:
import json
with open('results/tests_summary.json', 'w') as f:
    json.dump(data_dict, f)

In [ ]:
import json
with open('results/tests_summary.json', 'r') as f:
    data_dict = json.load(f)

data_dict

In [21]:
import pandas as pd
dataframe = pd.DataFrame()
for model, d in data_dict.items():
    sub_frame = pd.DataFrame(d)
    sub_frame['Model'] = model

    dataframe = pd.concat([dataframe, sub_frame], axis=0)
dataframe

,data_all,data_blogs,data_books,data_mails,Model
N=10 Accuracy,91.25% ± 7.69,90.60% ± 8.90,87.75% ± 10.08,28.65% ± 9.79,Model_all
N=10 Top-5 Accuracy,98.35% ± 3.47,98.25% ± 4.02,98.85% ± 3.31,68.85% ± 10.02,Model_all
N=20 Accuracy,86.60% ± 7.16,88.08% ± 6.68,81.45% ± 7.21,NaN,Model_all
N=20 Top-5 Accuracy,95.40% ± 4.28,96.70% ± 3.59,96.28% ± 3.86,NaN,Model_all
N=50 Accuracy,83.72% ± 4.60,82.51% ± 4.78,72.14% ± 5.34,NaN,Model_all
N=50 Top-5 Accuracy,93.73% ± 3.54,93.13% ± 3.58,91.94% ± 3.26,NaN,Model_all
N=100 Accuracy,78.39% ± 3.99,77.95% ± 3.95,65.07% ± 3.89,NaN,Model_all
N=100 Top-5 Accuracy,90.84% ± 2.55,90.14% ± 2.90,86.49% ± 2.81,NaN,Model_all
N=250 Accuracy,72.39% ± 2.39,71.56% ± 2.45,NaN,NaN,Model_all
N=250 Top-5 Accuracy,86.73% ± 2.07,85.77% ± 1.92,NaN,NaN,Model_all


In [25]:
print(dataframe.reset_index().set_index(['Model','index']).to_latex())

\begin{tabular}{llllll}
\toprule
              &                      &        data\_all &      data\_blogs &      data\_books &      data\_mails \\
Model & index &                 &                 &                 &                 \\
\midrule
Model\_all & N=10 Accuracy &   91.25\% ± 7.69 &   90.60\% ± 8.90 &  87.75\% ± 10.08 &   28.65\% ± 9.79 \\
              & N=10 Top-5 Accuracy &   98.35\% ± 3.47 &   98.25\% ± 4.02 &   98.85\% ± 3.31 &  68.85\% ± 10.02 \\
              & N=20 Accuracy &   86.60\% ± 7.16 &   88.08\% ± 6.68 &   81.45\% ± 7.21 &             NaN \\
              & N=20 Top-5 Accuracy &   95.40\% ± 4.28 &   96.70\% ± 3.59 &   96.28\% ± 3.86 &             NaN \\
              & N=50 Accuracy &   83.72\% ± 4.60 &   82.51\% ± 4.78 &   72.14\% ± 5.34 &             NaN \\
              & N=50 Top-5 Accuracy &   93.73\% ± 3.54 &   93.13\% ± 3.58 &   91.94\% ± 3.26 &             NaN \\
              & N=100 Accuracy &   78.39\% ± 3.99 &   77.95\% ± 3.95 &   65.07\% ± 3.89 

/tmp/ipykernel_1586346/2621851513.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(dataframe.reset_index().set_index(['Model','index']).to_latex())


In [18]:
from transformers import AutoModel
DEVICE = 1
REPEATS = 100
TOP_K = 5

n_list = [10, 20, 50, 100, 250]
keys = ['all', 'blogs', 'books', 'mails']

model = AutoModel.from_pretrained('roberta-large').cuda(DEVICE)

data_dict[f'Model_RoBERTa'] = {}
print(f'[log] Evaluating model - RoBERTa')

for k_data in keys:
    data = data_zoo[k_data]
        
    data_dict[f'Model_RoBERTa'][f'data_{k_data}'] = {}
    print(f'[log] Evaluating on data - {k_data}')

    for n in n_list:
        print(f'[log] Running {REPEATS} repetitions for N={n}')
        max_len = len(data.id.unique())
        
        if n == 'max':
            n = max_len

        if n <= max_len:
            acc, topk, acc_sd, topk_sd = evaluate(model, data, top_k=TOP_K, N=n, repetitions=REPEATS, embed_f=embed_transformer)
            data_dict[f'Model_RoBERTa'][f'data_{k_data}'][f'N={n} Accuracy'] = f'{100*acc:.2f}% ± {100*acc_sd:.2f}'
            data_dict[f'Model_RoBERTa'][f'data_{k_data}'][f'N={n} Top-5 Accuracy'] = f'{100*topk:.2f}% ± {100*topk_sd:.2f}'

            print(f'[log] model:RoBERTa_data:{k_data}_n:{n}: {100*acc:.2f}% {100*topk:.2f}%')

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[log] Evaluating model - RoBERTa
[log] Evaluating on data - all
[log] Running 100 repetitions for N=10


  0%|          | 0/100 [00:00<?, ?it/s]

[log] model:RoBERTa_data:all_n:10: 44.65% 76.55%
[log] Running 100 repetitions for N=20


  0%|          | 0/100 [00:00<?, ?it/s]

...

[log] model:RoBERTa_data:books_n:50: 28.91% 50.39%
[log] Running 100 repetitions for N=100


  0%|          | 0/100 [00:00<?, ?it/s]

[log] model:RoBERTa_data:books_n:100: 24.23% 42.11%
[log] Running 100 repetitions for N=250
[log] Evaluating on data - mails
[log] Running 100 repetitions for N=10


  0%|          | 0/100 [00:00<?, ?it/s]

[log] model:RoBERTa_data:mails_n:10: 23.85% 63.00%
[log] Running 100 repetitions for N=20
[log] Running 100 repetitions for N=50
[log] Running 100 repetitions for N=100
[log] Running 100 repetitions for N=250
